In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#!pip install ray[train]

In [3]:
#!pip install -U ipywidgets
# not needed for production, only for jupyter notebook

In [4]:
#--- need to run 
#memcached -m 10240 -I 20m -vv -u www-data &

In [ ]:
#--- should clean imports - copy paste
import ray
from ray.train import ScalingConfig
import torch
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
import ray.train.torch
import ray.util.state
import ray
import torch
import torch.nn as nn
import ray.train.torch
import ray.train
from torch.nn import CrossEntropyLoss, MSELoss
from torch.optim import Adam
from quickannotator.constants import TileStatus
from quickannotator.db import get_session
from quickannotator.db.models import Tile
import sqlalchemy
from datetime import datetime

/opt/uv_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-11 20:06:12,237	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 20:06:12,624	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 20:06:13,649	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [6]:
ray.init( namespace="quick_annotator")

2025-03-11 20:06:17,588	INFO worker.py:1752 -- Started a local Ray instance.


Python version:,3.10.12
Ray version:,2.10.0


(pid=68010) /opt/uv_venv/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
(pid=68010)   check_for_updates()


(DLActor pid=68010) actor_name='dlactor_2g'
(DLActor pid=68010) classid=2
(DLActor pid=68010) tile_size=2048
(DLActor pid=68010) starting up dlactor_2g
(DLActor pid=68010) 
(DLActor pid=68010) View detailed results here: /root/ray_results/TorchTrainer_2025-03-11_20-06-25
(DLActor pid=68010) To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2025-03-11_20-06-14_811208_66961/artifacts/2025-03-11_20-06-25/TorchTrainer_2025-03-11_20-06-25/driver_artifacts`


(DLActor pid=68010) [output] This will use the new output engine with verbosity 1. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
(DLActor pid=68010) InitSpatiaMetaData() error:"table spatial_ref_sys already exists"


(DLActor pid=68010) 
(DLActor pid=68010) Training started with configuration:
(DLActor pid=68010) ╭──────────────────────────────────────────────╮
(DLActor pid=68010) │ Training config                              │
(DLActor pid=68010) ├──────────────────────────────────────────────┤
(DLActor pid=68010) │ train_loop_config/actor_name      dlactor_2g │
(DLActor pid=68010) │ train_loop_config/classid                  2 │
(DLActor pid=68010) │ train_loop_config/magnification           10 │
(DLActor pid=68010) │ train_loop_config/tile_size             2048 │
(DLActor pid=68010) ╰──────────────────────────────────────────────╯


(TrainTrainable pid=68116) /opt/uv_venv/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
(TrainTrainable pid=68116)   check_for_updates()
(RayTrainWorker pid=68203) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=68116) Started distributed worker processes: 
(TorchTrainer pid=68116) - (ip=172.17.0.2, pid=68203) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=68203) [W311 20:06:36.662038104 Utils.hpp:136] Warning: Environment variable NCCL_ASYNC_ERROR_HANDLING is deprecated; use TORCH_NCCL_ASYNC_ERROR_HANDLING instead (function operator())


(RayTrainWorker pid=68203) actor_name='dlactor_2g'


(RayTrainWorker pid=68203) /opt/QuickAnnotator/quickannotator/dl/training.py:25: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
(RayTrainWorker pid=68203)   A.GaussNoise(p=.5, var_limit=(10.0, 50.0)),
(RayTrainWorker pid=68203) /opt/QuickAnnotator/quickannotator/dl/training.py:26: UserWarning: Argument(s) 'border_mode' are not valid for transform GridDistortion
(RayTrainWorker pid=68203)   A.GridDistortion(p=.5, num_steps=5, distort_limit=(-0.3, 0.3),
(RayTrainWorker pid=68203) /opt/QuickAnnotator/quickannotator/dl/training.py:30: UserWarning: Argument(s) 'eps' are not valid for transform RandomGamma
(RayTrainWorker pid=68203)   A.RandomGamma(p=.5, gamma_limit=(80, 120), eps=1e-07),
(RayTrainWorker pid=68203) /opt/uv_venv/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the en

(RayTrainWorker pid=68203) io_images going on GPU: 	 torch.Size([1, 3, 2112, 2112])
(RayTrainWorker pid=68203) saving annotations
(RayTrainWorker pid=68203) updating tile status


In [ ]:
from quickannotator.dl.training import train_pred_loop
# https://discuss.ray.io/t/core-how-to-make-sure-an-actor-is-initialized/1537/5 might be worth keeping 

@ray.remote
class DLActor:
    def __init__(self,actor_name,classid,tile_size,magnification):
        
        print(f"{actor_name=}")
        print(f"{classid=}")
        print(f"{tile_size=}")
        self.actor_name=actor_name #TODO: i don't know if there is a way to get this reflexively instead of passing it in? -- trying to get something working first : ) 
        self.classid=classid
        self.tile_size=tile_size
        self.enable_training=False # starts not training
        self.closedown = False 
        self.allow_pred=True # --- don't know if we'll ever need this, so hard setting to true
        self.hexid=None
        self.magnification = magnification

    def start_dlproc(self,allow_pred=True):
        #--------- this is a bit of a disaster - need to basically make sure we're not launching multple
        #TODO: check if hexid is alive and valid -- can't start two trainings!
        # if is_training(self.hexid):
        #     return False
        #need to check if is already training, if yes, no opt
        print("starting up", self.actor_name)
        #---------------------------

        #scaling_config = ray.train.ScalingConfig(num_workers=2, use_gpu=True,resources_per_worker={"GPU":.5})
        ## will fail on single node single gpu --- (DLActor pid=174521) Duplicate GPU detected : rank 1 and rank 0 both on CUDA device 1000

        #--- NEED A GPU CONTAINER TO TEST THIS        
        total_gpus  = ray.cluster_resources().get("GPU", 0)
        # total_gpus_available=  ray.available_resources().get("GPU", 0)
        # print(total_gpus,total_gpus_available)
        # resource_to_request = (total_gpus_available/total_gpus)/2 * 1.01
        # print(f"resource_to_request: {resource_to_request}")

        scaling_config = ray.train.ScalingConfig(num_workers=int(total_gpus), use_gpu=True,
                                                resources_per_worker={"GPU":.01}, placement_strategy="STRICT_SPREAD")
        
        #----
#        scaling_config = ray.train.ScalingConfig(num_workers=1, use_gpu=False)
        trainer = ray.train.torch.TorchTrainer(train_pred_loop,
                                       scaling_config=scaling_config,
                                       train_loop_config={'actor_name':self.actor_name,
                                                          'classid':self.classid,
                                                         'tile_size':self.tile_size,
                                                         'magnification':self.magnification})
        self.hexid = trainer.fit().hex() ##widly -- this doesn't save the result in the actor
        return self.hexid
    
    def infer(self,image_id,tileids): #only bulk should be performed, there is no difference between doing 1 versus 100 tiles, so this reduces code complexity
                                      #NOTE: there is another option here - that we accept ids from the tile table directly instead of having to accept both image_id and tile_id
        if not self.allow_pred:
            print("not doing inference --- actor was started with prediction disabled")
            return False
        # probably need to *Delete* the tiles that are associated with this tileid before executing the below, so that they're not duplicated
        # i would do that in a seperate function as "infer" doesn't logically mean "delete", so the expected behavior might be weird if it delets stuff unprompted
        # perhaps that could be clarified with a e.g., named function paramter - or perhaps a seperate function is really needed
        with get_session() as db_session:
            stmt = sqlalchemy.update(Tile).where(Tile.tile_id.in_(tileids), Tile.image_id == image_id,
                                                Tile.annotation_class_id == self.classid)\
                                                    .values(pred_status=TileStatus.STARTPROCESSING,pred_datetime=datetime.now()) ## should add date time

            # Execute the update
            db_session.execute(stmt)

        #need a similar statement to get the DL starting
        return True
    
    def getTileStatus(self,image_id,tile_ids): #probably belongs else where but need this for debug
        with get_session() as db_session:
            stmt = db_session.query(Tile).filter(Tile.tile_id.in_(tile_ids),Tile.image_id == image_id,
                            Tile.annotation_class_id == self.classid)

            result = stmt.all()
        
        return result

    def getClassId(self):
        return self.classid
        
    def getHexId(self):
        return self.hexid

    def setHexId(self,hexid:str): # not sure if set is smart to have -- should likely be done internally
        self.hexid=hexid
        return self.hexid

    def setEnableTraining(self,enable_training: bool):
        self.enable_training=enable_training
        return self.enable_training
    
    def getEnableTraining(self):
        return self.enable_training
    
    def getCloseDown(self):
        return self.closedown
    
    def setCloseDown(self,closedown: bool):
        self.closedown=closedown
        return self.closedown
    

    def getActorName(self):
        return self.actor_name
    
    def getTileSize(self):
        return self.tile_size


/opt/uv_venv/lib/python3.10/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.4'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


this section  is some non-ray testing to see if things work as expected. to use these, remove the @ray annotator in the actor above 

In [8]:
classid=2
actor_name=f"dlactor_{classid}g"
print(actor_name)
actor = DLActor.options(name=actor_name,lifetime="detached",max_concurrency=2).remote(actor_name=actor_name,classid=classid,
                                                                                      tile_size=2048,magnification=10)


dlactor_2g


In [9]:
ray.get(actor.getClassId.remote())

2

In [10]:
ray.get(actor.getTileSize.remote())

2048

In [11]:
ray.get(actor.getActorName.remote()) 

'dlactor_2g'

In [12]:
print(actor_name)

dlactor_2g


In [13]:
actor.setEnableTraining.remote(False)

ObjectRef(e0dc174c83599034f26fe9486c231d771b443d2e0100000001000000)

In [14]:
ref2=actor.start_dlproc.remote()
print(ref2)


ObjectRef(f4402ec78d3a2607f26fe9486c231d771b443d2e0100000001000000)


In [15]:
ray.get(actor.infer.remote(1,[100]))

True

In [16]:
asdfas!!!@3123

SyntaxError: invalid syntax (4093815909.py, line 1)

In [ ]:
actor.setEnableTraining.remote(True)

In [ ]:
ref2=actor.start_dlproc.remote()
print(ref2)



In [ ]:
asdfas!!!@3123

In [ ]:
actor.setEnableTraining.remote(False)

In [ ]:
ray.get(actor.infer.remote(1,[100]))

In [ ]:
##not ray test
class2=DLActor(classid=2)


In [ ]:
class2.infer(image_id=1,tileids=[323],status=1)

In [ ]:
# -- check it was set
tile=class2.getTileStatus([323])
tile[0].seen

In [ ]:
ref2=class2.start_dlproc()


This section is when the @Ray actor is enabled, and thus has remote. note we need max_concurrency = 2 for this to work

In [ ]:
class2 = DLActor.options(max_concurrency=2).remote(classid=2,tile_size=2_048)
ref2=class2.start_dlproc.remote()
print(ref2)

In [ ]:
print(ray.get(class2.getclassid.remote()))


In [ ]:
print(ray.get(class2.infer.remote(image_id=1,tileids=[323],status=1)))

In [ ]:
tile=(ray.get(class2.getTileStatus.remote([323])))
print(tile[0].seen)


In [ ]:
ref2=class2.start_dlproc.remote()
print(ref2)